In [1]:
import os
import sys
import codecs
import json
import argparse
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import twitter

sys.path.append("..")
import utils.global_function as global_func

In [2]:
config_path = '../config/twitter.yml'
token_path = '/data/twitter_token.yml'
results_dir = '/data/dust'
resume_search = False

In [47]:
"""load config"""
token_config = global_func.load_config(token_path)
config = global_func.load_config(config_path)

"""アクセストークンの読み込み."""
ConsumerKey = token_config['ConsumerKey']
ConsumerSecret = token_config['ConsumerSecret']
AccessToken = token_config['AccessToken']
AccessTokenSecret = token_config['AccessTokenSecret']

"""検索パラメータの読み込み"""
q = "。,！,？+-\n+-笑+-「+-」+-w+-ｗ+-(+-（+-http+-https+exclude:retweets"  # URL投稿やリツイートを含まないようにクエリ設定
search_params = config['search']
q = search_params.get('query', q)
geocode = search_params.get('geocode', '')
lang = search_params.get('lang', '')
locale = search_params.get('locale', '')
result_type = search_params.get('result_type', '')
count = search_params.get('count', 0)
until = search_params.get('until', '')
since_id = search_params.get('since_id', '')
max_id = search_params.get('max_id', '')
include_entities = search_params.get('include_entities', '')

"""検索パラメータ(非ポスト)の読み込み"""
search_count = search_params.get('search_count', 1)
search_flag = True

tweets_no = 1  # 保存時の通し番号
collected_data_count = 0  # 取得ツイート数

"""検索実行"""
t = twitter.Twitter(auth=twitter.OAuth(AccessToken, AccessTokenSecret, ConsumerKey, ConsumerSecret), retry=True)


try:
    datas = t.search.tweets(q=q, geocode=geocode, lang=lang, locale=locale,
                            result_type=result_type, count=count, until=until,
                            since_id=since_id, max_id=max_id, include_entities=include_entities)
    search_count -= 1
    if search_count == 0:
        search_flag = False

except Exception as e:
    print(e)

if datas["statuses"] == []:
    print("There is no tweet.")

In [48]:
datas.keys()

dict_keys(['statuses', 'search_metadata'])

In [49]:
datas['statuses']

[{'created_at': 'Sat Apr 06 11:39:27 +0000 2019',
  'id': 1114492812072853504,
  'id_str': '1114492812072853504',
  'text': '外カメで自撮りをする新しい技術を考えました🥺←気付いた優秀なフォロワーさんいるかな？\n\nあ、芋けんぴ髪の毛についてるよって言われたに使っていいよ🥺\n\n今日は僕の師匠の誕生日です、盛り上げるの手伝ってくれるいい子いませんか👶🏻… https://t.co/QcfoidCYOY',
  'truncated': True,
  'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1069302993521008640,
   'id_str': '1069302993521008640',
   'name': '結月 ゆの -universe-',
   'screen_name': 'un_yuno',
   'location': 'Universe',
   'description': "99' / ミナミ / ホスト / お酒は飲めません/ぴゅあぴゅあなアイドル営業/夢を超えるための予想や目標、考え方を投稿します🥺",
   'url': 'https://t.co/p5CmKcNxCh',
   'entities': {'url': {'urls': [{'url': 'https://t.co/p5CmKcNxCh',
       'expanded_url

In [50]:
def_tweet = datas['statuses'][1]


In [51]:
def del_dict_key(dict, del_keys):
    dict_ = dict.copy()
    for key in del_keys:
        del dict_[key]
    return dict_

def extract_dict_key(dict_, select_keys):
    del_keys = set(dict_.keys()) ^ set(select_keys)
    return del_dict_key(dict_, del_keys)

In [52]:
example_keys = ['created_at', 'id', 'user', 'text', 'retweet_count', 'favorite_count']

In [53]:
extract_dict_key(def_tweet, example_keys)

{'created_at': 'Thu Apr 04 13:18:49 +0000 2019',
 'id': 1113793044644368390,
 'text': '僕の優秀なフォロワーさん\nTwitterにこういう機能あったらいいのに\nみたいなんあります？',
 'user': {'id': 780396177606873088,
  'id_str': '780396177606873088',
  'name': 'Peach-gan(r6s初心者)',
  'screen_name': 'peachgan_r6s',
  'location': '',
  'description': 'youtubeで配信してます。\n自分ではよくわからないんですが\n暴言よく吐いているらしいです。\n\n嫌いでなければ、フォローお願いします。',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 62,
  'friends_count': 89,
  'listed_count': 0,
  'created_at': 'Mon Sep 26 13:18:42 +0000 2016',
  'favourites_count': 91,
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': False,
  'verified': False,
  'statuses_count': 787,
  'lang': 'ja',
  'contributors_enabled': False,
  'is_translator': False,
  'is_translation_enabled': False,
  'profile_background_color': 'F5F8FA',
  'profile_background_image_url': None,
  'profile_background_image_url_https': None,
  'profile_background_tile': False,
  'pr

In [63]:
def detect_username(dict_):
    usename = dict_.get('user', {}).get('screen_name', '')
    return usename

def detect_since_id(dict_):
    usename = dict_.get('id_str')
    return usename

In [64]:
usename = detect_username(def_tweet)

In [65]:
since_id = detect_since_id(def_tweet)

In [70]:
q = '@' + usename
try:
    datas = t.search.tweets(q=q, geocode=geocode, lang=lang, locale=locale,
                            result_type=result_type, count=100, until=until, since_id=since_id)

except Exception as e:
    print(e)

In [71]:
len(datas['statuses'])

10